In [ ]:
!pip install --upgrade scipy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import square, ShortTimeFFT
from scipy.signal.windows import gaussian

base_dir = "/kaggle/input/hms-harmful-brain-activity-classification"

fs = 200  # Sample rate.

df_traincsv = pd.read_csv(f'{base_dir}/train.csv')
print(df_traincsv.shape)
df_traincsv.head()


In [ ]:
# eeg_id = 722738444
# eeg_sub_id = 4

eeg_id = 1327593077
eeg_sub_id = 88

# eeg_id = 3988090520
# eeg_sub_id = 1

eeg = pd.read_parquet(f'{base_dir}/train_eegs/{eeg_id}.parquet')
print(eeg.shape)
eeg.head()

In [ ]:
rec = df_traincsv.loc[(df_traincsv.eeg_id == eeg_id) & (df_traincsv.eeg_sub_id == eeg_sub_id)].iloc[0]
rec

In [ ]:
# 50 second and center 10 second eeg sub samples 
offset = int(rec.eeg_label_offset_seconds)
start = offset * fs
end = (offset + 50) * fs
eeg_sub_50 = eeg[start:end]
start = (offset + 20) * fs
end = (offset + 30) * fs
eeg_sub_10 = eeg[start:end]

In [ ]:
def plot_eeg(ax, eeg, title, sep):
    srate = 200 # Sample rate.
    nx = eeg.shape[0]
    totaltime = nx/srate
    X, Y = np.linspace(0, totaltime, nx), np.zeros(nx)
    yticklabels = eeg.columns[::-1]  # Reversed.

    for i, label in enumerate(yticklabels):
        Y = eeg[label]
        ax.plot(X, Y + (i * sep), linewidth=0.5, color='black')

    ax.set_title(title)
    ax.set(ylim=(-0.5*sep, (len(yticklabels)-0.5)*sep),
           yticks=np.arange(len(yticklabels))*sep,
           yticklabels=yticklabels)
    ax.set_xlabel('time [s]')

fig, ax = plt.subplots(1, 1, figsize=(10, 15))

plot_eeg(ax, eeg_sub_10, title='10 seconds sample - eeg: ' + str(rec.eeg_id)
          + '/' + str(rec.eeg_sub_id) + ' ' + rec.expert_consensus, sep = 500)

plt.tight_layout()
plt.show()

Doing the spectrogram on a 50 s window.

In [ ]:
N = eeg_sub_50.shape[0]
t_x = np.arange(N) * 1/fs  # time indexes for signal
print(t_x[-1])
t_x.shape

In [ ]:
g_std = 16  # standard deviation for Gaussian window in samples
hop = 8  # 8
win_width = 49  # Pick an odd number.  49
win = gaussian(win_width, std=g_std, sym=True)  # symmetric Gaussian wind.
SFT = ShortTimeFFT(win, hop=hop, fs=fs, mfft=800)
electrode = 0
x = eeg_sub_50.iloc[:,electrode].values
Sx = SFT.spectrogram(x)  # calculate absolute square of STFT

# dt = hop*1/fs
t0 = win_width/(2*hop)
# 10 second window
ti = int(20*fs/hop + t0)
tf = int(30*fs/hop + t0)

n = int(30/SFT.delta_f)  # Number of bins below 30 Hz.
fig1, ax1 = plt.subplots(figsize=(6., 4.))  # enlarge plot a bit

ax1.imshow(Sx[1:(n+1), ti:tf], origin='lower', aspect='auto')

plt.show()
